In [5]:
import tensorflow as tf

# === Varsayılan davranış: GPU bellek kullanımını TensorFlow yönetir ===
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print("✅ GPU bulundu. Varsayılan bellek yönetimi kullanılacak.")
else:
    print("❌ GPU bulunamadı.")


✅ GPU bulundu. Varsayılan bellek yönetimi kullanılacak.


In [6]:
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_train_batch_begin(self, batch, logs=None):
        lr = self.model.optimizer.learning_rate
        if hasattr(lr, '__call__'):  # Eğer lr schedule ise
            lr = lr(self.model.optimizer.iterations)
        print(f"Step {self.model.optimizer.iterations.numpy()} - Learning Rate: {lr.numpy():.6f}")

In [8]:
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Optional: Enable mixed precision for GPUs with tensor cores
# from tensorflow.keras import mixed_precision
# mixed_precision.set_global_policy('mixed_float16')

# Fast approximate median using mean (significantly faster)
def reduce_median_fast(tensor, axis):
    return tf.reduce_mean(tensor, axis=axis)

# Optimized custom pooling layer
class CustomHDMRPooling2D(tf.keras.layers.Layer):
    def __init__(self, pool_size=(3, 3), strides=3):
        super().__init__()
        self.pool_size = pool_size
        self.strides = strides

    def call(self, inputs):
        k_h, k_w = self.pool_size
        stride = self.strides
        batch_size = tf.shape(inputs)[0]
        channels = tf.shape(inputs)[-1]

        patches = tf.image.extract_patches(
            images=inputs,
            sizes=[1, k_h, k_w, 1],
            strides=[1, stride, stride, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )

        out_h, out_w = tf.shape(patches)[1], tf.shape(patches)[2]
        patches = tf.reshape(patches, [batch_size, out_h, out_w, k_h, k_w, channels])

        row_sum = tf.reduce_sum(patches, axis=4)  # (B, OH, OW, H, C)
        col_sum = tf.reduce_sum(patches, axis=3)  # (B, OH, OW, W, C)
        f0 = tf.reduce_mean(patches, axis=[3, 4])

        f1 = row_sum/ tf.cast(k_w , tf.float32)  # (B, OH, OW, H)
        f2 = col_sum/ tf.cast(k_h , tf.float32)  # (B, OH, OW, W)
        
        f1_min=tf.reduce_min(f1,axis=3)
        f2_min=tf.reduce_min(f2,axis=3)

        f1_max=tf.reduce_max(f1,axis=3)
        f2_max=tf.reduce_max(f2,axis=3)

        f1_avg=tf.reduce_mean(f1,axis=3)
        f2_avg=tf.reduce_mean(f2,axis=3)

        f1_median=(f1_min+f1_max+f1_avg)/3.0
        f2_median=(f2_min+f2_max+f2_avg)/3.0


        scalar = f0+(f1_median+f2_median)/2.0
        return scalar

# === Load and preprocess CIFAR-10 ===
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# === Build model ===
model = tf.keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.Conv2D(32, kernel_size=3, activation='relu', padding='same'),
    CustomHDMRPooling2D(pool_size=(3, 3), strides=3),
    layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'),
    CustomHDMRPooling2D(pool_size=(3, 3), strides=3),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # Use dtype='float32' if using mixed_precision
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# === Train ===
model.fit(
    x_train, y_train,
    batch_size=128,  # Increased batch size
    epochs=10,
    validation_data=(x_test, y_test),
)

# === Evaluate ===
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")


Epoch 1/10
391/391 [==============================] - 11s 12ms/step - loss: 1.6847 - accuracy: 0.3894 - val_loss: 1.4517 - val_accuracy: 0.4670
Epoch 2/10
391/391 [==============================] - 4s 10ms/step - loss: 1.3830 - accuracy: 0.5042 - val_loss: 1.2853 - val_accuracy: 0.5431
Epoch 3/10
391/391 [==============================] - 4s 11ms/step - loss: 1.2617 - accuracy: 0.5535 - val_loss: 1.2238 - val_accuracy: 0.5663
Epoch 4/10
391/391 [==============================] - 4s 10ms/step - loss: 1.1710 - accuracy: 0.5885 - val_loss: 1.2003 - val_accuracy: 0.5658
Epoch 5/10
391/391 [==============================] - 4s 10ms/step - loss: 1.1114 - accuracy: 0.6093 - val_loss: 1.1034 - val_accuracy: 0.6141
Epoch 6/10
391/391 [==============================] - 4s 10ms/step - loss: 1.0654 - accuracy: 0.6254 - val_loss: 1.0656 - val_accuracy: 0.6253
Epoch 7/10
391/391 [==============================] - 4s 11ms/step - loss: 1.0279 - accuracy: 0.6382 - val_loss: 1.0366 - val_accuracy: 0.637